In [1]:
pip install dash plotly pandas

^C
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from dash import Dash, html, dash_table, dcc, callback, Output, Input
import plotly.express as px
import dash
from dash.dependencies import Input, Output
import plotly.graph_objs as go
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [ ]:
#loading the data
dementia_df = pd.read_csv('dementia.csv')

In [ ]:
dementia_df.head()

In [ ]:
#check columns with missing values
dementia_df.isna().any(axis=0).sum()

In [ ]:
#drop missing values
missing_values = dementia_df.isnull().sum()
print("\nMissing Values Count:")
print(missing_values)

In [ ]:
df_cleaned = dementia_df.dropna(axis=1)

In [ ]:
# Remove rows with any missing values
df_dementia_cleaned = df_cleaned.dropna()

In [ ]:
# Remove duplicate rows if any
dementia_cleaned = df_dementia_cleaned.drop_duplicates()

In [ ]:
missing1_values = dementia_cleaned.isnull().sum()
print("\nMissing Values Count:")
print(missing1_values)

In [ ]:
# Initialize the Dash app
app = Dash(__name__)

In [ ]:
# Define the layout of the dashboard
app.layout = html.Div([
    html.H1("Dementia Analysis Dashboard"),
    
    # Patient Data Visualization
    html.Div([
        html.H2("Patient Demographics"),
        dcc.Graph(id='age-distribution'),
        dcc.Graph(id='gender-distribution'),
        dcc.Graph(id='education-distribution')
    ]),
    
    # Cognitive Test Scores
    html.Div([
        html.H2("Cognitive Test Scores"),
        dcc.Graph(id='cognitive-scores')
    ]),
    
    # Comparative Analysis
    html.Div([
        html.H2("Comparative Analysis"),
        dcc.Dropdown(
            id='comparison-feature',
            options=[{'label': col, 'value': col} for col in dementia_cleaned.columns if dementia_cleaned[col].dtype == 'object'],
            value='Gender',
            style={'width': '50%'}
        ),
        dcc.Graph(id='comparison-plot')
    ]),
    
    # Predictive Analytics
    html.Div([
        html.H2("Predictive Analytics"),
        html.P("Model Accuracy Based on Selected Features:"),
        dcc.Dropdown(
            id='feature-selector',
            options=[{'label': col, 'value': col} for col in dementia_cleaned.columns if dementia_cleaned[col].dtype != 'object'],
            value=['Age', 'Cognitive_Test_Scores', 'AlcoholLevel'],
            multi=True,
            style={'width': '50%'}
        ),
        html.Div(id='prediction-output', style={'font-weight': 'bold', 'font-size': '16px', 'margin-top': '10px'})
    ]),
    
    # Care Quality Metrics
    html.Div([
        html.H2("Care Quality Metrics"),
        dcc.Graph(id='medication-history'),
        dcc.Graph(id='mri-delay')
    ])
])


In [ ]:
# Callback for age distribution
@app.callback(Output('age-distribution', 'figure'),
              Input('age-distribution', 'id'))
def update_age_distribution(_):
    fig = px.histogram(dementia_cleaned, x='Age', color='Dementia', 
                       title='Age Distribution by Dementia Status',
                       labels={'Dementia': 'Dementia Status'})
    return fig

In [ ]:
# Callback for gender distribution
@app.callback(Output('gender-distribution', 'figure'),
              Input('gender-distribution', 'id'))
def update_gender_distribution(_):
    gender_counts = dementia_cleaned.groupby(['Gender', 'Dementia']).size().reset_index(name='Count')
    fig = px.bar(gender_counts, x='Gender', y='Count', color='Dementia', 
                 title='Gender Distribution by Dementia Status',
                 labels={'Count': 'Count', 'Gender': 'Gender', 'Dementia': 'Dementia Status'})
    return fig

In [ ]:
# Callback for education distribution
@app.callback(Output('education-distribution', 'figure'),
              Input('education-distribution', 'id'))
def update_education_distribution(_):
    education_counts = dementia_cleaned.groupby(['Education_Level', 'Dementia']).size().reset_index(name='Count')
    fig = px.bar(education_counts, x='Education_Level', y='Count', color='Dementia', 
                 title='Education Level Distribution by Dementia Status',
                 labels={'Count': 'Count', 'Education_Level': 'Education Level', 'Dementia': 'Dementia Status'})
    return fig

In [ ]:
# Callback for cognitive scores
@app.callback(Output('cognitive-scores', 'figure'),
              Input('cognitive-scores', 'id'))
def update_cognitive_scores(_):
    fig = px.box(dementia_cleaned, x='Dementia', y='Cognitive_Test_Scores', 
                 title='Cognitive Test Scores by Dementia Status',
                 labels={'Dementia': 'Dementia Status', 'Cognitive_Test_Scores': 'Cognitive Test Scores'})
    return fig

In [ ]:
# Callback for comparative analysis
@app.callback(Output('comparison-plot', 'figure'),
              Input('comparison-feature', 'value'))
def update_comparison_plot(feature):
    fig = px.histogram(dementia_cleaned, x=feature, color='Dementia', 
                       title=f'Distribution of {feature} by Dementia Status',
                       labels={'Dementia': 'Dementia Status'})
    return fig

In [ ]:
# Callback for predictive analytics
@app.callback(Output('prediction-output', 'children'),
              Input('feature-selector', 'value'))
def update_prediction(selected_features):
    if not selected_features:
        return "Please select at least one feature."
    X = dementia_cleaned[selected_features]
    y = dementia_cleaned['Dementia']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    accuracy = model.score(X_test, y_test)
    return f"Random Forest Model Accuracy: {accuracy:.2f}"


In [ ]:
# Callback for medication usage
@app.callback(Output('medication-history', 'figure'),
              Input('medication-history', 'id'))
def update_medication_usage(_):
    medication_counts = dementia_cleaned['Medication_History'].value_counts().reset_index()
    # Rename the columns for clarity
    medication_counts.columns = ['Medication_History', 'Count']
    # Create the pie chart
    fig = px.pie(medication_counts, values='Count', names='Medication_History', 
                 title='Medication Usage Distribution')
    return fig

In [ ]:
# Callback for MRI delay
@app.callback(Output('mri-delay', 'figure'),
              Input('mri-delay', 'id'))
def update_mri_delay(_):
    fig = px.box(dementia_cleaned, x='Dementia', y='MRI_Delay', 
                 title='MRI Delay by Dementia Status',
                 labels={'Dementia': 'Dementia Status', 'MRI_Delay': 'MRI Delay (days)'})
    return fig


In [ ]:
# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

Result of the above visualization
1. Age distribution by Dementia status:
  This is a Histogram that visualizes the age distribution of patients in dementia status. The red color indicates patients diagonized with Dementia(categorized as 1) while the blue color indicates patients without dementia(categorized as 2). X-axis: Patients age ranging from 60-90 years. Y axis: Patients count in each age group. Distribution: patients with the age group 60-63 has higher count of dementia patients within age group 64-71 has higher count of no dementia Patients within the age group 72-77 has a higher count of Dementia Patients within the age group 79-87 has a higher count of no dementia Patients within the age group 88-91 has a higher count of Dementia From this Visualization, Age group with dementia varies and is not consistent. Therefore, increase in age does not highly significant to developing Dementia.

2. Gender Distribution by Dementia Status 
 This is a stacked barchart that visualizes Dementia prevalence across Gender. The yellow color indicates patients with dementia while blue color indicates patients gender without dementia. X axis: Gender (Male and Female) Y axis: Count of individual patients. From the observation, Female with dementia are 244 while males with dementia are 241. Also Females without dementia are 260 while males without dementia are 255. We can deduce that both genders have almost similar counts as the height of the bars are similar. The number of females with dementia is roughly equal to the number of males with dementia (Difference of 3 counts) and the number of females without dementia is roughly equals to the number of males without dementia (difference of 5 counts). Insights: Gender does not significantly affect the distribution of dementia status in the dataset as both genders ehibit theb same pattern.

3. Education level distribution by dementia status. 
  This visualization depicts the distribution of education levels by Dementia status. Each bars is separated by the education level. The yellow color indicates counts with dementia and blue indicates counts without dementia. X axis: This includes different education level like Diploma, No school, Primary school, and secondary school. Y axis : This includes the count of individuals in each category. From the observation, higher number of patients with diploma has no Dementia, Higher number of patients without school has Dementia Higher numvber of patients with primary school educational level has no dementia Higher number of patients with secondarty school has Dementia. This means the prevalence of dementia among patients with certain educational level varies.

4. Cognitive test scores by Dementia Status
This is a Boxplot visualization comparing cognitive test scores with two groups which are dementia represented as 1 and no dementia represented as 0. 
